In [1]:
import glob
import cv2
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor
import time

def process_image(img_path, output_folder, i, a):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    h, w = image.shape

    x_coords = np.arange(w)
    x_matrix = np.tile(x_coords, (h, 1))
    y_matrix = np.tile(np.arange(h).reshape(h, 1), (1, w))
    distance_matrix = np.abs(x_matrix - w / 2)
    mask = (y_matrix > a * distance_matrix ** 2).astype(np.uint8) * 255
    result = cv2.bitwise_and(image, image, mask=mask)

    cv2.imwrite(os.path.join(output_folder, '{}.jpeg'.format(i)), result)

# Input and output folders
input_folder = r"test"
output_folder = r"test_output"

# List of subfolders (NORMAL and PNEUMONIA)
subfolders = ['NORMAL', 'PNEUMONIA']

start_time = time.time()

# Process images for each subfolder using ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    for subfolder in subfolders:
        input_path = os.path.join(input_folder, subfolder)
        output_path = os.path.join(output_folder, subfolder)

        # Create the output subfolder if it doesn't exist
        os.makedirs(output_path, exist_ok=True)

        # Load and process images for the current subfolder in parallel
        image_paths = glob.glob(os.path.join(input_path, '*.jpeg'))
        params = [(img, output_path, i, 0.0025) for i, img in enumerate(image_paths)]
        executor.map(lambda p: process_image(*p), params)

end_time = time.time()

print(f"Execution time: {end_time - start_time} seconds")

Execution time: 28.31055212020874 seconds


In [2]:
import glob
import cv2
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor
import time

def process_image(img_path, output_folder, i, a):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    h, w = image.shape

    x_coords = np.arange(w)
    x_matrix = np.tile(x_coords, (h, 1))
    y_matrix = np.tile(np.arange(h).reshape(h, 1), (1, w))
    distance_matrix = np.abs(x_matrix - w / 2)
    mask = (y_matrix > a * distance_matrix ** 2).astype(np.uint8) * 255
    result = cv2.bitwise_and(image, image, mask=mask)

    cv2.imwrite(os.path.join(output_folder, '{}.jpeg'.format(i)), result)

# Input and output folders
input_folder = r"train"
output_folder = r"train_output"

# List of subfolders (NORMAL and PNEUMONIA)
subfolders = ['NORMAL', 'PNEUMONIA']

start_time = time.time()

# Process images for each subfolder using ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    for subfolder in subfolders:
        input_path = os.path.join(input_folder, subfolder)
        output_path = os.path.join(output_folder, subfolder)

        # Create the output subfolder if it doesn't exist
        os.makedirs(output_path, exist_ok=True)

        # Load and process images for the current subfolder in parallel
        image_paths = glob.glob(os.path.join(input_path, '*.jpeg'))
        params = [(img, output_path, i, 0.0025) for i, img in enumerate(image_paths)]
        executor.map(lambda p: process_image(*p), params)

end_time = time.time()

print(f"Execution time: {end_time - start_time} seconds")

Execution time: 213.2499566078186 seconds


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
img_width, img_height = 224, 224
num_of_class = 2

sample_train = keras. utils.image_dataset_from_directory(
    directory='train_output',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(img_width, img_height))

sample_test = keras. utils.image_dataset_from_directory(
    directory='test_output',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(img_width, img_height))

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [5]:
print(sample_train.class_names)

['NORMAL', 'PNEUMONIA']


In [6]:
# Extracting Features and Labels
x_train = []
y_train = []

for feature, label in sample_train:
    x_train.append(feature.numpy())
    y_train.append(label.numpy())
# Concatenate the lists to get the full 'x' and 'y' arrays
x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
print("done")

done


In [7]:
# Extracting Features and Labels
x_test = []
y_test = []

for feature, label in sample_test:
    x_test.append(feature.numpy())
    y_test.append(label.numpy())
# Concatenate the lists to get the full 'x' and 'y' arrays
x_test = np.concatenate(x_test, axis=0)
y_test = np.concatenate(y_test, axis=0)
print("done")

done


In [8]:
print("Shape of 'x_train':", x_train.shape)
print("Shape of 'y_train':", y_train.shape)
print("Shape of 'x_test':", x_test.shape)
print("Shape of 'y_test':", y_test.shape)

Shape of 'x_train': (5216, 224, 224, 3)
Shape of 'y_train': (5216, 2)
Shape of 'x_test': (624, 224, 224, 3)
Shape of 'y_test': (624, 2)


In [9]:
x_train=x_train/255
x_test = x_test/255

In [10]:
# from keras.layers.pooling.max_pooling1d import MaxPool1D
from sklearn.metrics import confusion_matrix
import itertools
# from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()
model.add(Conv2D(filters = 8, kernel_size = (5,5), padding = 'Same', activation= 'relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(224, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))

# Training The CNN
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
model.summary()
epochs = 30
# Visualized Layers of CNN
keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 8)       608       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 8)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 112, 112, 8)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 16)      1168      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 16)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 56, 56, 16)        0

In [11]:
hist = model.fit(x_train, y_train, epochs = epochs)

Epoch 1/30
163/163 [==============================] - 437s 2s/step - loss: 0.5688 - accuracy: 0.8092
Epoch 2/30
163/163 [==============================] - 165s 1s/step - loss: 0.2755 - accuracy: 0.8913
Epoch 3/30
163/163 [==============================] - 171s 1s/step - loss: 0.2481 - accuracy: 0.8982
Epoch 4/30
163/163 [==============================] - 372s 2s/step - loss: 0.2230 - accuracy: 0.9059
Epoch 5/30
163/163 [==============================] - 307s 2s/step - loss: 0.2058 - accuracy: 0.9156
Epoch 6/30
163/163 [==============================] - 177s 1s/step - loss: 0.1796 - accuracy: 0.9289
Epoch 7/30
163/163 [==============================] - 165s 1s/step - loss: 0.1726 - accuracy: 0.9373
Epoch 8/30
163/163 [==============================] - 155s 953ms/step - loss: 0.1640 - accuracy: 0.9377
Epoch 9/30
163/163 [==============================] - 156s 959ms/step - loss: 0.1603 - accuracy: 0.9388
Epoch 10/30
163/163 [==============================] - 157s 961ms/step - loss: 0.1426

In [12]:
Y_pred = model.predict(x_test)

20/20 [==============================] - 14s 234ms/step


In [13]:
def func(data,threshold):
  for i in range(len(data)):
      for j in range(len(data[i])):
          if data[i][j] < threshold:
              data[i][j] = 0
          else:
              data[i][j] = 1
  return data

In [14]:
Y_pred_01 = func(Y_pred,0.5)

In [15]:
Y_true = func(y_test,0.5)

In [16]:
from sklearn.metrics import classification_report, accuracy_score, multilabel_confusion_matrix

print(classification_report(Y_true,Y_pred_01))

              precision    recall  f1-score   support

           0       0.91      0.30      0.45       234
           1       0.70      0.98      0.82       390

   micro avg       0.73      0.73      0.73       624
   macro avg       0.80      0.64      0.63       624
weighted avg       0.78      0.73      0.68       624
 samples avg       0.73      0.73      0.73       624



In [17]:
from sklearn.metrics import accuracy_score
print("\n Accuracy of the model: {0}%".format(accuracy_score(Y_true, Y_pred_01)*100))


 Accuracy of the model: 72.59615384615384%
